In [1]:
import pandas as pd
from datetime import datetime
from clickhouse_driver import Client
import logging

def read_clickhouse_to_dataframe(host,port,user,password,database):
    query = 'SELECT email_address,first_name,last_name,address_1,address_2,postal_code,city,primary_phone_nbr,runid,source_name,file_name,model_nbr FROM registria'
    # Connect to ClickHouse server
    client1 = Client(host=host, port=port, user=user, password=password, database=database)
    try:
        # Execute SQL query to fetch data
        result = client1.execute(query)
        columns = ['email_address', 'first_name', 'last_name','address_1','address_2', 'postal_code', 'city', 'primary_phone_nbr', 'runid', 'source_name', 'file_name','model_nbr']
        df = pd.DataFrame(result, columns=columns)
        df=df.to_dict(orient="records")

    finally:
        # Close ClickHouse connection
        client1.disconnect()

    return df

def main_new(dfs,host,port,user,password,database):
    df = read_clickhouse_to_dataframe(host,port,user,password,database)
    for i in range(len(dfs)):
        try:
            dfs[i]['runid']=df[i]['runid']
            dfs[i]['zip_code']=df[i]['postal_code']
            dfs[i]['telephone']=df[i]['primary_phone_nbr']
            dfs[i]['last_name']=df[i]['last_name']
            dfs[i]['customer_name']=f"{df[i]['first_name']} {df[i]['last_name']}"
            dfs[i]['email']=df[i]['email_address']
            dfs[i]['city']=df[i]['city']
            dfs[i]['street_1']=df[i]['address_2']
            dfs[i]['street_2']=df[i]['address_1']
            dfs[i]['source']=df[i]['source_name']
            dfs[i]['file_name']=df[i]['file_name']
            dfs[i]['insertion_date']=str(datetime.now().date())
            dfs[i]['material']=df[i]['model_nbr']
        except Exception as e:
            logging.info(e)
        
    return dfs